In [ ]:
from datasets import load_dataset

def create_conversation(sample):
    system_message = ""

    if sample["messages"][0]["role"] == "system":
        return sample
    else:
        sample["messages"] = [{"role": "system", "content": system_message}] + sample["messages"]
        return sample
    
dataset = load_dataset("HuggingFaceH4/no_robots")

columns_to_remove = list(dataset["train"].featurers)
columns_to_remove.remove("messages")
dataset = dataset.map(create_conversation, remove_columns=columns_to_remove, batched=False)

dataset["train"] = dataset["train"].filter(lambda x: len(x["messages"][1:]) % 2 == 0)
dataset["test"] = dataset["test"].filter(lambda x: len(x["messages"][1:]) % 2 == 0)

dataset["train"].to_json("train_dataset.json", orient="records", force_ascii=False)
dataset["test"].to_json("test_dataset.json", orient="records", force_ascii=False)

In [3]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model_id = "./llama-3-8b-hf-no-robot"

model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    torch_dtype=torch.float16,
    quantization_config={"load_in_4bit": True},
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)